In [434]:
from PIL import Image, ImageDraw
import random
import numpy as np

In [466]:
# read image
im = Image.open('test_cap.png')
Vsize, Hsize = im.size
VnumArray = np.arange(Vsize)
HnumArray = np.arange(Hsize)
boxLen = 0
boxWid = 0
print Vsize, Hsize
im.show()

1920 960


In [468]:
# convert image data to numpy format
imPIX = im.load()
imDATA = np.zeros((3, Hsize, Vsize))

for x in range(Hsize):
    for y in range(Vsize):
        imDATA[0][x,y] = imPIX[y,x][0]
        imDATA[1][x,y] = imPIX[y,x][1]
        imDATA[2][x,y] = imPIX[y,x][2]

In [469]:
# quantization
def quantization(data, stat=np.array([0,85,170,255])):
    delta = data - stat
    delta = abs(delta)
    mindelta = delta.min()
    return stat[delta == mindelta]

for x in range(Hsize):
    for y in range(Vsize):
        imDATA[0][x,y] = quantization(imDATA[0][x,y])
        imDATA[1][x,y] = quantization(imDATA[1][x,y])
        imDATA[2][x,y] = quantization(imDATA[2][x,y])
## test
#data = np.array([78])
#stat=np.array([0,85,170,255])
#delta = data - stat
#delta = abs(delta)
#mindelta = delta.min()
#stat[delta == mindelta]

In [470]:
# correct

In [471]:


imDATA[0].mean(axis=0)

array([ 10.00520833,  10.00520833,  10.09375   , ...,  10.00520833,
        10.00520833,  10.00520833])

In [472]:
a = imDATA[1].mean(axis=1)


In [475]:
# cal, LtUpPt,LtDnPt,LtDnPt,RtDnPt,X0,Y0,X1,Y1
#boxLen
#boxWid

# white row/col is belivable
whiteRow = HnumArray[imDATA[0].mean(axis=1) > 225]
whiteCol = VnumArray[imDATA[0].mean(axis=0) > 225]
# black is uncertainty because of the captured boarder
blackRow = HnumArray[imDATA[0].mean(axis=1) < 25]
blackCol = VnumArray[imDATA[0].mean(axis=0) < 25]

if whiteRow.shape == whiteCol.shape:
    boxLen = whiteRow.shape[0]/4
else:
    raise Exception ("ERROR: whiteRow is not equal to whiteCol")

print whiteRow, whiteCol, blackRow
Y0 = whiteCol[2*boxLen-1] + boxLen + 1
Y1 = whiteCol[2*boxLen]   - boxLen
X0 = whiteRow[2*boxLen-1] + boxLen + 1
X1 = whiteRow[2*boxLen]   - boxLen

LtUpPt = (X0, Y0)
LtDnPt = (X1, Y0)
RtUpPt = (X0, Y1)
RtDnPt = (X1, Y1)

if (X1-X0)%boxLen !=0 or (Y1-Y0)%boxLen !=0 :
    raise Exception("(X1-X0) mod boxLen = %d, (Y1-Y0) mod boxLen=%d" % ((X1-X0)%boxLen, (Y1-Y0)%boxLen))


Exception: (X1-X0) mod boxLen = 4, (Y1-Y0) mod boxLen=5

In [ ]:
# check and correct each box
whiteRow

In [ ]:
# get pixel of each box
def getBoxPIX(imPayload, boxNum=(0,0), boxLen=4):
    boxData = np.zeros([3,boxLen, boxLen])
    i0 = boxNum[0]*boxLen
    i1 = (boxNum[0]+1)*boxLen
    j0 = boxNum[1]*boxLen
    j1 = (boxNum[1]+1)*boxLen
    boxData[0] = imPayload[0][i0:i1,j0:j1]
    boxData[1] = imPayload[1][i0:i1,j0:j1]
    boxData[2] = imPayload[2][i0:i1,j0:j1]
    print boxData
    return boxData[0].mean(), boxData[1].mean(), boxData[2].mean()


In [474]:
imPayload = np.zeros([3, X1-X0, Y1-Y0])
imPayload[0] = imDATA[0][X0:X1, Y0:Y1]
imPayload[1] = imDATA[1][X0:X1, Y0:Y1]
imPayload[2] = imDATA[2][X0:X1, Y0:Y1]
print imPayload.shape
getBoxPIX(imPayload, boxNum=(20,5), boxLen=boxLen)

(3L, 886L, 1846L)
[[[ 170.  170.  170.   85.   85.   85.   85.]
  [ 170.  170.  170.   85.   85.   85.   85.]
  [ 170.  170.  170.   85.   85.   85.   85.]
  [  85.   85.   85.  170.  170.  170.  170.]
  [  85.   85.   85.  170.  170.  170.  170.]
  [  85.   85.   85.  170.  170.  170.  170.]
  [  85.   85.   85.  170.  170.  170.  170.]]

 [[ 170.  170.  170.   85.   85.   85.   85.]
  [ 170.  170.  170.   85.   85.   85.   85.]
  [ 170.  170.  170.   85.   85.   85.   85.]
  [ 170.  170.  170.  255.  255.  255.  255.]
  [ 170.  170.  170.  255.  255.  255.  255.]
  [ 170.  170.  170.  255.  255.  255.  255.]
  [ 170.  170.  170.  255.  255.  255.  255.]]

 [[ 255.  255.  255.  255.  255.  255.  255.]
  [ 255.  255.  255.  255.  255.  255.  255.]
  [ 170.  170.  170.  170.  170.  170.  170.]
  [  85.   85.   85.   85.   85.   85.   85.]
  [   0.    0.    0.    0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.    0.    0.]
  [   0.    0.    0.    0.    0.    0.    0.]]]


(128.36734693877551, 176.9387755102041, 109.28571428571429)

In [444]:
boxLen

7L

In [445]:
imPIX[19,19]

(254, 0, 0)

In [446]:
imPIX[21,21]

(254, 0, 0)

In [447]:
imPIX[22,22]

(254, 0, 0)

In [448]:
imPIX[23,23]

(254, 0, 0)

In [346]:
imPIX[24,24]

(161, 61, 173)

In [347]:
imPIX[25,25]

(255, 0, 255)

In [ ]:
def isBlackLine():
    pass
def isWhiteLine():
    pass
def isTimingLine():
    pass
def isRtUpCorn():
    pass
def isRtDnCorn():
    pass
def isLtUpCorn():
    pass
def isLtDnCorn():
    pass
def getBoxLen():
    pass
def getPayloadRtUpPoint():
    pass
def getPayloadRtDnPoint():
    pass
def getPayloadLtUpPoint():
    pass
def getPayloadLtDnPoint():
    pass
def getBoxValue():
    pass

In [261]:
# depth=3, row=4, column=2
data = np.zeros((3,4,2))
print data
for i in range(4):
    for j in range(2):
        r = random.choice([1,2,3,4])
        g = random.choice([1,2,3,4])
        b = random.choice([1,2,3,4])
        r, g, b = 255/r, 255/g, 255/b
        data[0][i][j] = r
        data[1][i][j] = g
        data[2][i][j] = b
print data

[[[ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]]

 [[ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]]

 [[ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]]]
[[[  63.  255.]
  [  85.   85.]
  [ 127.  255.]
  [  85.  255.]]

 [[  63.  127.]
  [ 255.  255.]
  [  63.   63.]
  [ 255.   63.]]

 [[ 127.  255.]
  [ 255.  255.]
  [ 255.   63.]
  [ 255.  255.]]]


In [300]:
data[1]=data[1][0,1]


In [308]:
data[2][0:1,0:1]

array([[ 127.]])